# Проект SQL.
## Анализ базы данных приложения для чтения

### Описание проекта

###### Общая информация:
Компания купила крупный сервис для чтения книг по подписке. 

###### Цель:
Проанализировать базу данных для дальнейшего формулирования ценностного предложения для нового продукта.

###### Задачи:
- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;
- Определить автора с самой высокой средней оценкой книг — учитывая только книги с 50 и более оценками;
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

#### Описание данных
**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

### 1. Исследование данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 


Загрузим все необходимые таблицы:

In [2]:
query = ''' SELECT *
            FROM books
        ''' 

books = pd.io.sql.read_sql(query, con = engine, index_col = 'book_id')
books.sample()

,author_id,title,num_pages,publication_date,publisher_id
book_id,,,,,
152,359,Circus of the Damned (Anita Blake Vampire Hun...,320,2007-01-02,45


In [3]:
books.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 1 to 1000
Data columns (total 5 columns):
author_id           1000 non-null int64
title               1000 non-null object
num_pages           1000 non-null int64
publication_date    1000 non-null object
publisher_id        1000 non-null int64
dtypes: int64(3), object(2)
memory usage: 46.9+ KB


In [4]:
query = ''' SELECT *
            FROM authors
        ''' 

authors = pd.io.sql.read_sql(query, con = engine, index_col = 'author_id')
authors.sample()

,author
author_id,
480,Philip Roth


In [5]:
authors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 636 entries, 1 to 636
Data columns (total 1 columns):
author    636 non-null object
dtypes: object(1)
memory usage: 9.9+ KB


In [6]:
query = ''' SELECT *
            FROM publishers
        ''' 

publishers = pd.io.sql.read_sql(query, con = engine, index_col = 'publisher_id')
publishers.sample()


,publisher
publisher_id,
221,Perennial / William Morrow / HarperCollins


In [7]:
publishers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 340 entries, 1 to 340
Data columns (total 1 columns):
publisher    340 non-null object
dtypes: object(1)
memory usage: 5.3+ KB


In [8]:
query = ''' SELECT *
            FROM ratings
        ''' 

ratings = pd.io.sql.read_sql(query, con = engine, index_col = 'rating_id')
ratings.sample()

,book_id,username,rating
rating_id,,,
3587,581,johnsonamanda,5


In [9]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6456 entries, 1 to 6456
Data columns (total 3 columns):
book_id     6456 non-null int64
username    6456 non-null object
rating      6456 non-null int64
dtypes: int64(2), object(1)
memory usage: 201.8+ KB


In [10]:
query = ''' SELECT *
            FROM reviews
        ''' 

reviews = pd.io.sql.read_sql(query, con = engine, index_col = 'review_id')
reviews.sample()


,book_id,username,text
review_id,,,
1556,565,julie81,Amount conference throughout cost return natur...


In [11]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2793 entries, 1 to 2793
Data columns (total 3 columns):
book_id     2793 non-null int64
username    2793 non-null object
text        2793 non-null object
dtypes: int64(1), object(2)
memory usage: 87.3+ KB


### 3.1. Посчитаем, сколько книг вышло после 1 января 2000 года:

In [12]:
query = ''' SELECT COUNT(book_id) as books_cnt
            FROM books
            WHERE
            publication_date >= '2000-01-01'
        ''' 
books_since_2000  = pd.io.sql.read_sql(query, con = engine)
books_since_2000

,books_cnt
0,821


C 2000 года была выпущена 821 книга.

### 3.2. Для каждой книги посчитаем количество обзоров и среднюю оценку;

In [30]:
query = ''' 
            SELECT 
                DISTINCT ratings.book_id AS book_id,
                AVG(ratings.rating) AS rating_avg,
                COUNT(DISTINCT reviews.review_id) AS reviews_cnt
                
            FROM
                ratings
            INNER JOIN
                reviews ON reviews.book_id = ratings.book_id
            GROUP BY ratings.book_id


        ''' 
my_q= pd.io.sql.read_sql(query, con = engine)
my_q#.reviews_cnt.sum()

,book_id,rating_avg,reviews_cnt
0,807,4.000000,4
1,639,3.500000,1
2,287,3.500000,2
3,64,4.230769,4
4,889,3.500000,2
...,...,...,...
989,747,2.500000,2
990,38,4.000000,3
991,536,4.500000,2
992,559,3.666667,2


### 3.3. Определим издательство, которое выпустило наибольшее число книг толще 50 страниц

In [ ]:
query = ''' SELECT 
                publishers.publisher AS publisher,
                COUNT(books.book_id) AS books_cnt
            FROM 
                books
            INNER JOIN publishers ON publishers.publisher_id = books.publisher_id
            WHERE books.num_pages >= 50
            GROUP BY
                publishers.publisher
            ORDER BY books_cnt DESC
            LIMIT 3

        ''' 
my_q= pd.io.sql.read_sql(query, con = engine)
my_q

Ниже еще одно проверочное решение с подзапросами:

In [ ]:
query3 = ''' 

            SELECT 
                publishers.publisher AS publisher, Sub.books_cnt
            FROM publishers
            INNER JOIN
                (SELECT 
                    books.publisher_id AS publisher_id, 
                    COUNT(books.book_id) AS books_cnt
                FROM books
                WHERE books.num_pages >= 50
                GROUP BY books.publisher_id
                ) AS Sub ON Sub.publisher_id = publishers.publisher_id
            GROUP BY publishers.publisher, Sub.books_cnt
            ORDER BY Sub.books_cnt DESC
            LIMIT 3

            
        ''' 
my_q= pd.io.sql.read_sql(query, con = engine)
my_q

Издатель-лидер по количеству выпущенных книг - Penguin Books (42 книги).

### 3.4. Определим автора с самой высокой средней оценкой книг (учитывая только книги с кол-вом оценок больше 50)

In [ ]:
query = '''  
            SELECT Sub3.author, AVG(Sub3.rating) AS rating
            FROM
                (SELECT 
                    Sub2.author,
                    Sub1.rating
                FROM
                        (SELECT 
                        authors.author AS author,
                        books.book_id AS book_id
                    FROM authors
                    INNER JOIN books ON books.author_id = authors.author_id
                    WHERE books.book_id IN
                        (SELECT 
                                ratings.book_id AS book_id 
                            FROM ratings
                            GROUP BY ratings.book_id
                            HAVING COUNT(ratings.rating_id) >= 50)
                    GROUP BY authors.author, books.book_id) AS Sub2
                INNER JOIN 
                        (SELECT 
                        ratings.book_id AS book_id,
                        AVG(ratings.rating) AS rating
                    FROM ratings
                    WHERE ratings.book_id IN
                        (SELECT 
                            ratings.book_id AS book_id 
                        FROM ratings
                        GROUP BY ratings.book_id
                        HAVING COUNT(ratings.rating_id) >= 50)
                    GROUP BY ratings.book_id) AS Sub1 ON Sub2.book_id = Sub1.book_id
                GROUP BY Sub2.author, Sub1.rating
                ORDER BY Sub1.rating DESC) AS Sub3
            GROUP BY Sub3.author
            ORDER BY rating DESC
            ''' 
my_q= pd.io.sql.read_sql(query, con = engine)
my_q

J.K. Rowling/Mary GrandPré	 - автор с самой высокой оценкой книг.

### 3.5. Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [ ]:

query = ''' 
            SELECT AVG(Sub.reviews_cnt)
            FROM
                (SELECT 
                    reviews.username AS username,
                    COUNT(reviews.review_id) AS reviews_cnt
                FROM reviews
                WHERE reviews.username IN

                    (SELECT 
                        ratings.username AS username
                    FROM ratings
                    GROUP BY ratings.username
                    HAVING COUNT(ratings.rating_id) >= 50)
                GROUP BY reviews.username) AS Sub
      

        ''' 


my_q= pd.io.sql.read_sql(query, con = engine)
my_q

В среднем пользователи, которые поставили больше 50 оценок, написали 24 ревью.